# This notebook was used to create the subsea inpainting dataset

### Imports

In [ ]:
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import shutil
from tqdm import tqdm

### Code to extract frames from videos

In [ ]:
class Video:
    def __init__(self, filepath):
        self._filepath = Path(filepath)
        self._cap = cv2.VideoCapture(filepath.as_posix())
        self._video_info()
        
    def _video_info(self):
        self._total_frames = int(self._cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self._frame_rate = self._cap.get(cv2.CAP_PROP_FPS)
        # Duration in seconds
        self._duration = self._total_frames / self._frame_rate
        
    def __repr__(self):
        s = []
        s.append(f'Video file: {self._filepath.name}')
        s.append(f'Total Frames: {self._total_frames}')
        s.append(f'Frame Rate: {self._frame_rate:.2f} frames per second')
        s.append(f'Duration: {self._duration:.1f} seconds')
        return '\n'.join(s)
    
    def save_frames(self, output_folder, max_frames=None):
        output_folder = Path(output_folder)
        output_folder.mkdir(parents=True, exist_ok=True)
        base_name = self._filepath.name.split('.')[0]
        frames_to_extract = max_frames if max_frames else self._total_frames
        for frame_index in tqdm(range(frames_to_extract)):
            self._cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
            success, frame = self._cap.read()
            if success:
                filename = f'{base_name}_{frame_index:07d}.png'
                filepath = (output_folder / filename).as_posix()
                cv2.imwrite(filepath, frame)
            

# Step 1: Choose a video, pick a letter for the clip name and run the code cells below to extract the frames

In [ ]:
video_name = '05'
clip_name = video_name + 'f'

base_folder = '/home/bms/devops/subsea_inpainting/dataset'
video_input_folder = Path(base_folder) / 'original_videos' / video_name
frames_outuput_folder = Path(base_folder) / 'prepared_images' / clip_name / 'frames'

In [ ]:
video_filepath = list(video_input_folder.glob('*.mp4'))[0]
print(video_filepath)

In [ ]:
video = Video(video_filepath)
video

In [ ]:
video.save_frames(frames_outuput_folder, max_frames=500)

# Step 2: Delete unwanted frames from folder
 * Use any image visualization software to choose which frames to delete. I used Xviewer
 
# Step 3: Draw a mask template on an external software
 * Use any image editing software with layer support to that. I used GIMP.
 * White pixels means inpaint. Black pixels means keep content.
 
# Step 4: Run the code below to copy the mask template
 * Some inpainting methods needs one mask file per frame file, even if the mask is the same

In [ ]:
mask_template_folder = Path(base_folder) / 'prepared_images' / clip_name / 'mask_template'
frames_folder = Path(base_folder) / 'prepared_images' / clip_name / 'frames'
masks_folder = Path(base_folder) / 'prepared_images' / clip_name / 'masks'

In [ ]:
mask_template_folder

In [ ]:
template_filepath = list(mask_template_folder.glob('*.png'))[0]
print(template_filepath)

In [ ]:
masks_folder.mkdir(parents=True, exist_ok=True)

for frame in tqdm(frames_folder.glob('*.png')):
    mask_filepath = masks_folder / frame.name
    shutil.copy(template_filepath, mask_filepath)